### Gerekli Kütüphaneler

In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px
import plotly.graph_objs as go

### Veriseti Hakkında
* Major Event Day (`../Datasets/med.csv`): Gün içindeki elektrik kesinti süresinin kabul edilebilir limiti aştığı gün. Dolayısıyla kesinti olduysa o gün tahmin edilen enerjinin sapmasının yüksek olmasını bekleriz. Çünkü kesinti nedeniyle dağıtılamamış.  

* Dağıtılan Enerji (Watt): Aktif(yararlı) gücün birimi.  

* MWh = MegaWatt saat  

> Dağıtılan Enerji'nin %85'i İzmir, %15'i ise Manisa için gerçekleştirilmektedir.



elektrik kesintisi günleri:

In [2]:
med_path = '../Datasets/med.csv'
med = pd.read_csv(med_path)
med['Tarih'] = pd.to_datetime(med['Tarih']);med.head()

,Tarih
0,2019-12-23
1,2019-06-12
2,2019-01-25
3,2019-09-25
4,2019-03-29


örnek submission:

In [3]:
sub_path = '../Datasets/sample_submission.csv'
sample_submission = pd.read_csv(sub_path);sample_submission.head()

,Tarih,Dağıtılan Enerji (MWh)
0,2022-08-01 00:00:00,0.0
1,2022-08-01 01:00:00,0.0
2,2022-08-01 02:00:00,0.0
3,2022-08-01 03:00:00,0.0
4,2022-08-01 04:00:00,0.0


<hr>

### Verilerin Keşfi

In [4]:
train_path = '../Datasets/train.csv'
df = pd.read_csv(train_path);df.head()

,Tarih,Dağıtılan Enerji (MWh)
0,2018-01-01 00:00:00,1593.944216
1,2018-01-01 01:00:00,1513.933887
2,2018-01-01 02:00:00,1402.612637
3,2018-01-01 03:00:00,1278.527266
4,2018-01-01 04:00:00,1220.697701


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40152 entries, 0 to 40151
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Tarih                   40152 non-null  object 
 1   Dağıtılan Enerji (MWh)  40152 non-null  float64
dtypes: float64(1), object(1)
memory usage: 627.5+ KB


In [6]:
df['Tarih'] = pd.to_datetime(df['Tarih']);df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40152 entries, 0 to 40151
Data columns (total 2 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   Tarih                   40152 non-null  datetime64[ns]
 1   Dağıtılan Enerji (MWh)  40152 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 627.5 KB


In [7]:
df.head(3)

,Tarih,Dağıtılan Enerji (MWh)
0,2018-01-01 00:00:00,1593.944216
1,2018-01-01 01:00:00,1513.933887
2,2018-01-01 02:00:00,1402.612637


In [8]:
df['Saat'] = df['Tarih'].dt.hour
df['Yılın Günleri'] = df['Tarih'].dt.day_of_year
df['Haftanın Günleri'] = df['Tarih'].dt.day_name()
df['Ay'] = df['Tarih'].dt.month
df['Yıl'] = df['Tarih'].dt.year
df['Hafta'] = df['Tarih'].dt.week;df.head(3)

C:\Users\ramaz\AppData\Local\Temp\ipykernel_6452\2329916610.py:6: FutureWarning:

Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.



,Tarih,Dağıtılan Enerji (MWh),Saat,Yılın Günleri,Haftanın Günleri,Ay,Yıl,Hafta
0,2018-01-01 00:00:00,1593.944216,0,1,Monday,1,2018,1
1,2018-01-01 01:00:00,1513.933887,1,1,Monday,1,2018,1
2,2018-01-01 02:00:00,1402.612637,2,1,Monday,1,2018,1


<hr>

### Verilerin Görselleştirilmesi

Saatlik verilerin incelenmesi

In [9]:
temp = df.iloc[:,:2]

fig = px.line(data_frame=temp,
              x='Tarih', y='Dağıtılan Enerji (MWh)',
              title='Saate göre Dağıtılan Enerji Miktarı')
fig.show()

Günlük verilerin incelenmesi

In [21]:
# Kesintileri göstermek için
temp = df.iloc[:,[0,1]]
kesintiler = med.copy()
kesintiler['Kesinti'] = 1
temp = pd.merge(temp, kesintiler, on='Tarih', how='outer')
temp.fillna(0, inplace= True)
temp['Tarih'] = temp['Tarih'].dt.date
temp = temp.groupby('Tarih').sum().reset_index()

fig = px.bar(data_frame=temp,
             x='Tarih', y='Dağıtılan Enerji (MWh)',
             title='Kesintili Günlere göre Dağıtılan Toplam Enerji Miktarı')

fig.add_trace(go.Scatter(x=temp[temp['Kesinti']==1]['Tarih'],
                        y=temp[temp['Kesinti']==1]['Dağıtılan Enerji (MWh)'],
                        mode='markers', name='Kesinti'))

fig.show()

In [12]:
temp = df.iloc[:,[0,1,4]].groupby('Haftanın Günleri')['Dağıtılan Enerji (MWh)'].mean().reset_index()

weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
temp['Haftanın Günleri'] = pd.Categorical(temp['Haftanın Günleri'], categories=weekdays, ordered=True)

temp = temp.sort_values('Haftanın Günleri')

fig = px.bar(data_frame=temp,
            x='Haftanın Günleri', y='Dağıtılan Enerji (MWh)',
            title='Haftanın Günlerine göre Dağıtılan Enerji Miktarı',
            color='Haftanın Günleri')

fig.add_trace(px.line(data_frame=temp,
                    x='Haftanın Günleri',
                    y='Dağıtılan Enerji (MWh)',
                    markers=True).data[0])
fig.show() 

Haftalık verilerin incelenmesi 

In [13]:
temp = df.iloc[:,[1,-1]].groupby('Hafta')['Dağıtılan Enerji (MWh)'].sum().reset_index()

fig = px.line(data_frame=temp,
              x='Hafta', y='Dağıtılan Enerji (MWh)',
              title='Haftalara göre Dağıtılan Toplam Enerji Miktarı')
fig.show()

In [14]:
temp = df[(df['Tarih']>='2021-01-01 00:00:00')&(df['Tarih']<='2021-12-31 23:59:00')]
temp = temp.iloc[:,[1,-1]].groupby('Hafta')['Dağıtılan Enerji (MWh)'].sum().reset_index()

fig = px.line(data_frame=temp,
              x='Hafta', y='Dağıtılan Enerji (MWh)',
              title='2021 Yılı Haftalara göre Dağıtılan Enerji Miktarı')
fig.show()

In [15]:
temp = df[(df['Tarih']>='2020-01-01 00:00:00')&(df['Tarih']<='2020-12-31 23:59:00')]
temp = temp.iloc[:,[1,-1]].groupby('Hafta')['Dağıtılan Enerji (MWh)'].sum().reset_index()

fig = px.line(data_frame=temp,
              x='Hafta', y='Dağıtılan Enerji (MWh)',
              title='2020 Yılı Haftalara göre Dağıtılan Enerji Miktarı')
fig.show()

Aylık verilerin incelenmesi

In [16]:
temp = df.iloc[:,[1,-3,-2]].groupby(['Ay','Yıl'])['Dağıtılan Enerji (MWh)'].sum().reset_index()

fig = px.bar(data_frame=temp,
        x='Ay', y='Dağıtılan Enerji (MWh)',
        title='Aylara göre Dağıtılan Enerji Miktarı',
        color='Yıl')
fig.show()

Yıllık verilerin incelenmesi

In [17]:
temp = df.iloc[:,[0,1,-2]].groupby('Yıl')['Dağıtılan Enerji (MWh)'].sum().reset_index()

fig = px.bar(data_frame=temp,
            x='Yıl', y='Dağıtılan Enerji (MWh)',
            title='Yıllara göre Dağıtılan Enerji Miktarı')

fig.show()

<hr>